
# 1. Train and evaluate two different multiple regression models using different optimization algorithms. Describe your approach, interpret the performance, and justify which machine learning model you would choose.

In [1]:
import pandas as pd

# Load the Bike Shop dataset from Git
url = 'https://raw.githubusercontent.com/VAWi-DataScience/Data-Science-and-Machine-Learning/main/Assignment/WiSe22_23/02_Bike%20rental.csv'
bikeRentalDf = pd.read_csv(url)
bikeRentalDf

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


In [2]:
# Import libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error

# Preprocess the dataset
bikeRentalDf = bikeRentalDf.drop(['instant', 'dteday', 'casual', 'registered'], axis=1)

# Split the dataset into training and testing sets
X = bikeRentalDf.drop('cnt', axis=1)
y = bikeRentalDf['cnt']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train two different multiple regression models using different optimization algorithms
model1 = LinearRegression()
model1.fit(X_train, y_train)

model2 = SGDRegressor()
model2.fit(X_train, y_train)

# Evaluate the performance of the models and compare them
y_pred1 = model1.predict(X_test)
mse1 = mean_squared_error(y_test, y_pred1)

y_pred2 = model2.predict(X_test)
mse2 = mean_squared_error(y_test, y_pred2)

print('MSE Model 1:', mse1)
print('MSE Model 2:', mse2)

# Choose the best model and justify the choice
# The model with the lower MSE is the better model. In this case, we would choose the model with the lower MSE as our best model. We can also consider other performance metrics such as R^2 score, but MSE is a commonly used metric to evaluate regression models.


MSE Model 1: 691035.0082022652
MSE Model 2: 862379.3268848669


In [4]:
print(bikeRentalDf)


     season  yr  mnth  holiday  weekday  workingday  weathersit      temp  \
0         1   0     1        0        6           0           2  0.344167   
1         1   0     1        0        0           0           2  0.363478   
2         1   0     1        0        1           1           1  0.196364   
3         1   0     1        0        2           1           1  0.200000   
4         1   0     1        0        3           1           1  0.226957   
..      ...  ..   ...      ...      ...         ...         ...       ...   
726       1   1    12        0        4           1           2  0.254167   
727       1   1    12        0        5           1           2  0.253333   
728       1   1    12        0        6           0           2  0.253333   
729       1   1    12        0        0           0           1  0.255833   
730       1   1    12        0        1           1           2  0.215833   

        atemp       hum  windspeed   cnt  
0    0.363625  0.805833   0.1604